In [2]:
import os
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

from sklearn.model_selection import ParameterGrid
from time import time

# Generate data

In [3]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(128, noise_strength=noise_strength)
validation_data = get_lorenz_data(2, noise_strength=noise_strength)

In [4]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3 #
params['model_order'] = 1
params['poly_order'] = 3 #
params['include_sine'] = False #
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0 #
params['loss_weight_sindy_z'] = 0.0 #
params['loss_weight_sindy_x'] = 1e-4 #
params['loss_weight_sindy_regularization'] = 1e-5 #

params['activation'] = 'sigmoid' #
params['widths'] = [64,32] #

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 128 #
params['learning_rate'] = 1e-2 #

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = False
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 100
params['refinement_epochs'] = 20


In [5]:
def dftocsv(df):
    if os.path.isfile('gridsearch_results.csv'):
        df.to_csv('gridsearch_results.csv', mode='a', header=False)
    else:
        df.to_csv('gridsearch_results.csv', mode='w', header=True)

# Set up model and training parameters

In [6]:
param_grid = {
    'latent_dim': [1, 3, 6],
    'loss_weight_sindy_z': [0.0, 1e-4],
    'loss_weight_sindy_x': [0.0, 1e-4],
    'widths' : [[64],[64, 32],[64,32,16]],
    'batch_size': [128, 32]
}
grid = ParameterGrid(param_grid)



# Run training experiments

In [7]:
for i, changed_params in enumerate(grid):
    params_run = params.copy()
    params_run.update(changed_params)
    params_run['coefficient_mask'] = np.ones((params_run['library_dim'], params_run['latent_dim']))
    
    print(f'EXPERIMENT:{i+1}')
    start = time()
    results_dict = train_network(training_data, validation_data, params_run)
    end = time()
    dt = end-start

    df = df.append({**results_dict, **params_run}, ignore_index=True)

    df2save = df[['loss_decoder','loss_sindy_x', 'loss_sindy_z' 'loss_sindy_regularization','validation_losses']]
    params_run['widths'] = str(params_run['widths'])
    df_params = pd.DataFrame(params_run, index=[0])

    df_final = pd.concat([df2save, df_params], axis=1)
    df_final['dt'] = dt

    dftocsv(df_final)

print("DONE")

EXPERIMENT:1


AttributeError: module 'tensorflow' has no attribute 'placeholder'